In [ ]:
from azureml.core import Workspace, Model
ws= Workspace.from_config("./config")
new_run= ws.get_run("d1295ce7-3e98-4de9-a302-3ad271adbe55")
new_run.register_model(model_path='outputs/models.pkl',
                        model_name='MS_POC_Titanic_Model',
                        tags={'source':'sdk-run','algorithm':'LogisticRegression'},
                        description="Microsoft POC for Titanic Dataset")

In [ ]:
from azureml.core import Workspace, Model
ws= Workspace.from_config("./config")
new_run= ws.get_run("df204f6d-b25f-4b1f-85f7-8f45b0cbe60b")
print(new_run)

In [ ]:
models=Model.list(ws)
for model in Model.list(ws):
    print("\n",model.name, 'version: ',model.version)
    print("model run id ",model.run_id)
    print("model name: ",model.name)

In [ ]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies   

In [ ]:
myenv= Environment(name="MyEnvironment")
myenv_dep= CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults'])
myenv.python.conda_dependencies = myenv_dep
myenv.register(ws)

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget
cluster_name= 'mspoctitanicaks'
if cluster_name not in ws.compute_targets:
    aks_config = AksCompute.provisioning_configuration(location='eastus',
                                                        vm_size='STANDARD_D11_V2',
                                                        agent_count=1,
                                                        cluster_purpose='DevTest')
    production_cluster= ComputeTarget.create(ws, cluster_name, aks_config)
    production_cluster.wait_for_completion(show_output=True)
    cluster= ws.compute_targets[cluster_name]
else:
    cluster= ws.compute_targets[cluster_name]
    print("cluster found here...")

In [ ]:
from azureml.core.model import InferenceConfig
infernece_config = InferenceConfig(environment=myenv,
                                    entry_script='scoring_script.py',
                                    source_directory='kubernetes_script/')

In [ ]:
from azureml.core.webservice import AksWebservice
deploy_config = AksWebservice.deploy_configuration(cpu_cores=1,memory_gb=1,enable_app_insights=True)

In [ ]:
model=ws.models['MS_POC_Titanic_Model']

In [ ]:
service = Model.deploy(workspace=ws,
                        name='ms-poc-titanic-webservice',
                        models=[model],
                        inference_config=infernece_config,
                        deployment_config=deploy_config,
                        deployment_target=cluster)
service.wait_for_deployment(show_output=True)